# Feature Engineering

* num_private: 0 if x =0, 1, otherwise
* amount_sth: 0 if x =0, 1, otherwise    
* population: 0-0, 1-1, 2 otherwise    
* construction_year: replace 0 with Nan, convert to age and normalize 
* longitude : 0 if < 5 and 1 otherwise

In [ ]:
#Replace 0 with nan
import numpy as np
print(train['construction_year'].head())

train.construction_year.replace(0, np.nan, inplace=True);
all_data.construction_year.replace(0, np.nan, inplace=True);

print(train['construction_year'].head())

In [ ]:
#Covert to age
print(train['construction_year'].head())

train['age'] =train['construction_year'].apply(lambda x: 2020-x);
all_data['age'] =all_data['construction_year'].apply(lambda x: 2020-x);

print(train['age'].head())

train.drop('construction_year', axis=1, inplace=True);
all_data.drop('construction_year', axis=1, inplace=True);

In [ ]:
#Encode amount_sth
print(train['amount_tsh'].head())

train['amount_coded'] =train['amount_tsh'].apply(lambda x: 0 if x==0.0 else 1);
all_data['amount_coded'] =all_data['amount_tsh'].apply(lambda x: 0 if x==0.0 else 1);

print(train['amount_coded'].head())

train.drop('amount_tsh', axis=1, inplace=True);
all_data.drop('amount_tsh', axis=1, inplace=True);

In [ ]:
#Encode num_private
print(train['num_private'].head())

train['num_coded'] =train['num_private'].apply(lambda x: 0 if x==0 else 1);
all_data['num_coded'] =all_data['num_private'].apply(lambda x: 0 if x==0 else 1);

print(train['num_coded'].head())

train.drop('num_private', axis=1, inplace=True);
all_data.drop('num_private', axis=1, inplace=True);

In [ ]:
#Encode logitude
print(train['longitude'].head())

train['longitude_coded'] =train['longitude'].apply(lambda x: 0 if x<5 else 1);
all_data['longitude_coded'] =all_data['longitude'].apply(lambda x: 0 if x<5 else 1);

print(train['longitude_coded'].head())

train.drop('longitude', axis=1, inplace=True);
all_data.drop('longitude', axis=1, inplace=True);

In [ ]:
#Encode population
print(train['population'].head())

train['population_coded'] =train['population'].apply(lambda x: 0 if x==0 else (1 if x==1 else 2));
all_data['population_coded'] =all_data['population'].apply(lambda x: 0 if x==0 else (1 if x==1 else 2));

print(train['population_coded'].head())

train.drop('population', axis=1, inplace=True);
all_data.drop('population', axis=1, inplace=True);

In [ ]:
pd.pivot_table(train, index = 'label', values = ['age','longitude_coded','num_coded', 'amount_coded', 'population_coded'])

In [ ]:
#Convert to categorical 
train.longitude_coded=train.longitude_coded.astype("object");
train.num_coded=train.num_coded.astype("object");
train.amount_coded=train.amount_coded.astype("object");
train.population_coded=train.population_coded.astype("object");

all_data.longitude_coded=all_data.longitude_coded.astype("object");
all_data.num_coded=all_data.num_coded.astype("object");
all_data.amount_coded=all_data.amount_coded.astype("object");
all_data.population_coded=all_data.population_coded.astype("object");

### Null Values

In [ ]:
train.isnull().mean().round(4) * 100

In [ ]:
#impute nulls for continuous data 
train.age = train.age.fillna(train.age.mean());
all_data.age = all_data.age.fillna(all_data.age.mean());

In [ ]:
na_cat_col = ['funder', 'installer', 'subvillage', 'public_meeting', 'permit']

In [ ]:
#impute nulls for categorical data 
from sklearn.impute import SimpleImputer

for col in na_cat_col:
    imputer1 =SimpleImputer(strategy='most_frequent')
    imputer1.fit(train[[col]])
    train[col] = imputer1.transform(train[[col]]).ravel();

In [ ]:
#impute nulls for categorical data 
from sklearn.impute import SimpleImputer

for col in na_cat_col:
    imputer1 =SimpleImputer(strategy='most_frequent')
    imputer1.fit(all_data[[col]])
    all_data[col] = imputer1.transform(all_data[[col]]).ravel();

In [ ]:
train.isnull().mean().round(4) * 100

In [ ]:
#drop scheme_name
train.drop('scheme_name', axis = 1, inplace=True);
all_data.drop('scheme_name', axis = 1, inplace=True);

In [ ]:
# Extract numeric and categorical columns
numerical_cols = [cname for cname in train.columns if train[cname].dtype in ['int64', 'float64']]
categorical_cols = [cname for cname in train.columns if train[cname].dtype == "object" and train[cname].nunique() <20]

print(numerical_cols)
print(categorical_cols)

### Preprocessing

In [ ]:
#created dummy variables from categories (also can use OneHotEncoder)
all_dummies = pd.get_dummies(all_data[['gps_height', 'latitude', 'region_code', 'train_test', 'age',
                                       'basin', 'public_meeting', 'permit', 'extraction_type_class', 
                                       'management_group', 'payment_type', 'quality_group', 'quantity_group', 
                                       'source_class', 'waterpoint_type_group', 
                                       'amount_coded', 'num_coded', 'longitude_coded', 'population_coded']])

In [ ]:
all_dummies.head()

In [ ]:
#Split to train test again
X_train = all_dummies[all_dummies.train_test == 1].drop(['train_test'], axis =1)
X_test = all_dummies[all_dummies.train_test == 0].drop(['train_test'], axis =1)


y_train = all_data[all_data.train_test==1].label

In [ ]:
# Scale data 
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
all_dummies_scaled = all_dummies.copy()
all_dummies_scaled[numerical_cols]= scale.fit_transform(all_dummies_scaled[numerical_cols])


X_train_scaled = all_dummies_scaled[all_dummies.train_test == 1].drop(['train_test'], axis =1)
X_test_scaled = all_dummies_scaled[all_dummies.train_test == 0].drop(['train_test'], axis =1)

In [ ]:
all_dummies_scaled.head()